In [1]:
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_ADDR
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_PORT

In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer
from ml.models.base.v2.brisk_xgboost import BriskXGBoost
from ml.models.base.v2.slug_xgboost import SlugXGBoost
from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_rf import SlugRF
from ml.models.base.slug_knn import SlugKNN
from ml.models.base.v2.brisk_bagging import BriskBagging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
import ray
import time
import pandas as pd

In [4]:
rayer.get_global_cluster(num_cpus=4)

In [5]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-24 02:15:47.232181 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.905 GiB object_store_memory

Demands:
 {'CPU': 1}: 4+ from request_resources()


In [8]:
ray.autoscaler.sdk.request_resources(num_cpus=0)

In [5]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [6]:
X_train_id = ray.put(X_train)
y_train_id = ray.put(y_train)
X_test_id = ray.put(X_test)
y_test_id = ray.put(y_test)


In [7]:

@ray.remote(num_returns=1)
def worker(base_model, X_train, X_test, y_train, y_test):     
    base_model.fetch_model(X_train, X_test, y_train, y_test)
    return base_model


In [12]:
brisk_xgb1 = BriskXGBoost('brisk_xgb1')
brisk_xgb1.boosted_round = 100
brisk_xgb1.n_trials = 1000

brisk_xgb2 = BriskXGBoost('brisk_xgb2')
brisk_xgb2.boosted_round = 150
brisk_xgb2.n_trials = 500

base_models = [brisk_xgb1, brisk_xgb2]

brisk_bagging_1 = BriskBagging('brisk_bagging_1')
brisk_bagging_1.boosted_round = 100
brisk_bagging_1.n_trials = 50

brisk_bagging_2 = BriskBagging('brisk_bagging_2')
brisk_bagging_2.boosted_round = 100
brisk_bagging_2.n_trials = 50

base_models_bagging = [brisk_bagging_1, brisk_bagging_2]

slug_xgb1 = SlugXGBoost('slug_xgb1')
slug_xgb1.boosted_round = 100
slug_xgb1.n_trials = 500

#slug_xgb2 = SlugXGBoost('slug_xgb2', X_train, X_test, y_train, y_test)
#slug_xgb2.boosted_round = 10
#slug_xgb2.n_trials = 10

#base_models_slug = [slug_xgb1, slug_xgb2]#

#slug_ann_1 = SlugANN('slug_ann_1', X_train, X_test, y_train, y_test)
#slug_ann_1.epochs = 50
#slug_ann_1.n_trials = 50

#slug_ann_2 = SlugANN('slug_ann_2', X_train, X_test, y_train, y_test)
#slug_ann_2.epochs = 50
#slug_ann_2.n_trials = 50

#base_models_ann = [slug_ann_1, slug_ann_2]


#slug_rf_1 = SlugRF('slug_rf_1', X_train, X_test, y_train, y_test)
#slug_rf_1.max_n_estimators = 1000
#slug_rf_1.n_trials = 500

#slug_rf_2 = SlugRF('slug_rf_2', X_train, X_test, y_train, y_test)
#slug_rf_2.max_n_estimators = 1000
#slug_rf_2.n_trials = 500

#base_models_rf = [slug_rf_1, slug_rf_2]


#slug_knn_1 = SlugKNN('slug_knn_1', X_train, X_test, y_train, y_test)
#slug_knn_1.n_neighbors = 100
#slug_knn_1.n_trials = 5000

#slug_knn_2 = SlugKNN('slug_knn_2', X_train, X_test, y_train, y_test)
#slug_knn_2.n_neighbors = 100
#slug_knn_1.n_trials = 5000

#base_models_knn = [slug_knn_1, slug_knn_2]

model_list = [slug_xgb1, brisk_xgb1, brisk_xgb2]


In [13]:
model_results = ray.get([worker.remote(base_model, X_train_id, X_test_id, y_train_id, y_test_id) for base_model in model_list])

(worker pid=130, ip=10.10.37.125) 2022-11-24T00:38:54PST : INFO : font_manager : _load_fontmanager : 1633 : Message : generated new fontManager
(worker pid=131, ip=10.10.20.33) 2022-11-24T00:38:54PST : INFO : font_manager : _load_fontmanager : 1633 : Message : generated new fontManager
(worker pid=589) 2022-11-24T00:38:54PST : INFO : brisk_xgboost : __discover_model__ : 109 : Message : brisk_xgb2: Starting training for trials:500, boosted rounds: 150, max depth: 10
(worker pid=589) /tmp/ray/session_2022-11-24_00-30-35_328980_8/runtime_resources/pip/6359c2ba15c7e71f0e73b7159cb8c36699f062e1/virtualenv/lib/python3.9/site-packages/optuna/_optimize.py:93: UserWarning: The default storage cannot be shared by multiple processes. Please use an RDB (RDBStorage) when you use joblib for multi-processing. The usage of RDBStorage can be found in https://optuna.readthedocs.io/en/stable/tutorial/rdb.html.
(worker pid=589)   warnings.warn(
(worker pid=131, ip=10.10.20.33) 2022-11-24T00:38:54PST : INFO

In [19]:
model_results[0].score

[0.8738940636546498, 0.7867130839181712]

In [36]:
model_results[1].score

[0.014120463031467745, -0.0830211453548515]

In [16]:
model_results[1].predict(X)

array([1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486196,
       1018.42486196, 1018.42486196, 1018.42486196, 1018.42486